In [25]:
import pandas
import COVID19Py

In [62]:


def parse_api_json(json, source):
    for_pandas = []
    confirmed = []
    deaths = []
    recovered = []
    for location in json['locations']:
        timeline_entry = {}
        entry = {
            'id': location['id'],
            'lat': location['coordinates']['latitude'],
            'lon': location['coordinates']['longitude'],
            'Date': pandas.to_datetime(location['last_updated']),
            'province': location['province'],
            'country_code': location['country_code'],
            'country': location['country'],
            'confirmed': location['latest']['confirmed'],
            'deaths': location['latest']['deaths'],
            'recovered': location['latest']['recovered'],
            'county': ''}
        if 'county' in location.keys():
            entry['county'] = location['county']
        if 'state' in location.keys():
            entry['state'] = location['state']
        entry['source'] = source
        for_pandas.append(entry)

        if 'timelines' in location.keys():
            for status in location['timelines']:
                for date in location['timelines'][status]['timeline']:
                    sub_entry = {'id': location['id'],
                                 'Date': pandas.to_datetime(date),
                                 status: location['timelines'][status]['timeline'][date]}
                    if status == 'confirmed':
                        confirmed.append(sub_entry)
                    elif status == 'deaths':
                        deaths.append(sub_entry)
                    else:
                        recovered.append(sub_entry)
    if confirmed:
        # return confirmed,deaths,recovered
        timeline_df = pandas.DataFrame(confirmed)
        if deaths:
            timeline_df = timeline_df.merge(
                pandas.DataFrame(deaths), on=['id', 'Date'])
        else:
            timeline_df['deaths'] = 0
        if recovered:
            timeline_df = timeline_df.merge(
                pandas.DataFrame(recovered), on=['id', 'Date'])
        else:
            timeline_df['recovered'] = 0.0

        main_df = pandas.DataFrame(for_pandas)
        timeline_df = timeline_df.merge(
            main_df[['id', 'lat', 'lon', 'province', 'country_code', 'country', 'source']], on=['id'])

        return main_df, timeline_df
    else:
        return pandas.DataFrame(for_pandas)

covid19_csbs = COVID19Py.COVID19(data_source="csbs").getAll(timelines=True)
covid19_jhu = COVID19Py.COVID19(data_source="jhu").getAll(timelines=True)
jhu_df, jhu_df_time = parse_api_json(covid19_jhu, 'JHU')
csbs_df = parse_api_json(covid19_csbs, 'CSBS')

In [67]:
sub_df = jhu_df_time[jhu_df_time[group] != '']
province_mapper = sub_df[
    sub_df['Date'] == official_date].groupby(
        group, as_index=False).apply(lambda x: x.sort_values('confirmed')[::-1].head(1)[[group, 'id', 'lat', 'lon']])
sub_df = sub_df[
    sub_df['Date'] == official_date].groupby(group, as_index=False).sum()[['province', 'confirmed', 'deaths', 'recovered']].merge(province_mapper, on=['province'])

##Now do it for the CSBS
province_mapper = csbs_df.groupby(
        group, as_index=False).apply(lambda x: x.sort_values('confirmed')[::-1].head(1)[[group, 'id', 'lat', 'lon']])
sub_df_csbs = csbs_df.groupby(group, as_index=False).sum()[['province', 'confirmed', 'deaths', 'recovered']].merge(province_mapper, on=['province'])

In [77]:
per_day_stats_by_state = pd.read_csv(
    'https://jordansdatabucket.s3-us-west-2.amazonaws.com/covid19data/provence_df_per_day.csv.gz', index_col=0)

In [20]:
# Import from S3:
jhu_df = pd.read_csv(
    'https://jordansdatabucket.s3-us-west-2.amazonaws.com/covid19data/jhu_df.csv.gz', index_col=0)
jhu_df_time = pd.read_csv(
    'https://jordansdatabucket.s3-us-west-2.amazonaws.com/covid19data/jhu_df_time.csv.gz', index_col=0)
csbs_df = pd.read_csv(
    'https://jordansdatabucket.s3-us-west-2.amazonaws.com/covid19data/csbs_df.csv.gz', index_col=0)
per_day_stats_by_country = pd.read_csv(
    'https://jordansdatabucket.s3-us-west-2.amazonaws.com/covid19data/country_df_per_day.csv.gz', index_col=0)
per_day_stats_by_state = pd.read_csv(
    'https://jordansdatabucket.s3-us-west-2.amazonaws.com/covid19data/provence_df_per_day.csv.gz', index_col=0)

jhu_df_time['Date'] = pd.to_datetime(jhu_df_time['Date'])
jhu_df['Date'] = pd.to_datetime(jhu_df['Date'])
csbs_df['Date'] = pd.to_datetime(csbs_df['Date'])

date_mapper = pd.DataFrame(
    jhu_df_time['Date'].unique(), columns=['Date'])
date_mapper['Date_text'] = date_mapper['Date'].dt.strftime('%m/%d/%y')
min_date = date_mapper.index[0]
max_date = date_mapper.index[-1]


In [80]:
official_date = date_mapper.iloc[-1]['Date']

In [90]:
from datetime import date, timedelta


official_date.date() == date.today() - timedelta(days=1)

True

In [89]:
date.yesterday()

AttributeError: type object 'datetime.date' has no attribute 'yesterday'

In [7]:
official_date = date_mapper['Date'].iloc[-1]
country_mapper = jhu_df_time[jhu_df_time['Date'] == official_date].groupby(
    'country', as_index=False).apply(lambda x: x.sort_values('confirmed')[::-1].head(1)[['country', 'id', 'lat', 'lon']])
sub_df = jhu_df_time[jhu_df_time['Date'] == official_date].groupby(
    'country', as_index=False).sum().merge(country_mapper, on=['country'])
   

In [13]:
group='province'
sub_df = jhu_df_time[jhu_df_time[group] != '']
province_mapper = sub_df[
    sub_df['Date'] == official_date].groupby(
        group, as_index=False).apply(lambda x: x.sort_values('confirmed')[::-1].head(1)[[group, 'id', 'lat', 'lon']])
sub_df = sub_df[
    sub_df['Date'] == official_date].groupby(group, as_index=False).sum()[['province', 'confirmed', 'deaths', 'recovered']].merge(province_mapper, on=['province'])


In [19]:
jhu_df_time[jhu_df_time['country'] == 'US']['province']

13950    NaN
13951    NaN
13952    NaN
13953    NaN
13954    NaN
        ... 
14007    NaN
14008    NaN
14009    NaN
14010    NaN
14011    NaN
Name: province, Length: 62, dtype: object

In [ ]:
import warnings
import pandas as pd
import glob
import os
import logging
import boto3
from botocore.exceptions import ClientError
import COVID19Py
import pandas

# Cancel copy warnings of pandas
warnings.filterwarnings(
    "ignore", category=pd.core.common.SettingWithCopyWarning)


covid19_csbs = COVID19Py.COVID19(data_source="csbs").getAll(timelines=True)
covid19_jhu = COVID19Py.COVID19(data_source="jhu").getAll(timelines=True)


def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(
            file_name, bucket, object_name,  ExtraArgs={'ACL': 'public-read'})
    except ClientError as e:
        logging.error(e)
        return False
    return True


def parse_api_json(json, source):
    for_pandas = []
    confirmed = []
    deaths = []
    recovered = []
    for location in json['locations']:
        timeline_entry = {}
        entry = {
            'id': location['id'],
            'lat': location['coordinates']['latitude'],
            'lon': location['coordinates']['longitude'],
            'Date': pandas.to_datetime(location['last_updated']),
            'province': location['province'],
            'country_code': location['country_code'],
            'country': location['country'],
            'cases': location['latest']['confirmed'],
            'deaths': location['latest']['deaths'],
            'recovered': location['latest']['recovered'],
            'county': ''}
        if 'county' in location.keys():
            entry['county'] = location['county']
        if 'state' in location.keys():
            entry['state'] = location['state']
        entry['source'] = source
        for_pandas.append(entry)

        if 'timelines' in location.keys():
            for status in location['timelines']:
                for date in location['timelines'][status]['timeline']:
                    sub_entry = {'id': location['id'],
                                 'Date': pandas.to_datetime(date),
                                 status: location['timelines'][status]['timeline'][date]}
                    if status == 'confirmed':
                        confirmed.append(sub_entry)
                    elif status == 'deaths':
                        deaths.append(sub_entry)
                    else:
                        recovered.append(sub_entry)
    if confirmed:
        #return confirmed,deaths,recovered
        timeline_df = pandas.DataFrame(confirmed)
        if deaths:
            timeline_df = timeline_df.merge(
                pandas.DataFrame(deaths), on=['id', 'Date'])
        else:
            timeline_df['deaths'] = 0
        if recovered:
            timeline_df = timeline_df.merge(
                pandas.DataFrame(recovered), on=['id', 'Date'])
        else:
            timeline_df['recovered'] = 0.0
        
        main_df = pandas.DataFrame(for_pandas)
        timeline_df = timeline_df.merge(
            main_df[['id', 'lat', 'lon', 'province', 'country_code', 'country', 'source']], on=['id'])

        return main_df, timeline_df
    else:
        return pandas.DataFrame(for_pandas)
    
jhu_df, jhu_df_time = parse_api_json(covid19_jhu, 'JHU')
csbs_df = parse_api_json(covid19_csbs, 'CSBS')


In [ ]:
covid19_jhu['locations'][0]['latest']:

In [ ]:
covid19_jhu['locations'][0]['timelines']

In [ ]:
import COVID19Py

covid19_csbs = COVID19Py.COVID19(data_source="csbs").getAll(timelines=True)
covid19_jhu = COVID19Py.COVID19(data_source="jhu").getAll(timelines=True)

In [ ]:
covid19_jhu

In [ ]:
for i in [i.keys() for i in covid19_csbs['locations']]:
    if "timelines" in i:
        print(i)
        break

In [ ]:
jhu_df[jhu_df['province'] != '']

In [ ]:
covid19_jhu['locations'][0]

In [ ]:
import requests
URL = "https://coronavirus-tracker-api.herokuapp.com/"
LOCATIONS_COUNTRY = "v2/locations?timelines=1"
LOCATIONS_COUNTY = "v2/locations?timelines=1&source=csbs"

locations_county = requests.get(URL+LOCATIONS_COUNTY).json()
#locations_country = requests.get(URL+LOCATIONS_COUNTRY).json()


In [ ]:
def parse_api_json(json,source):
    for_pandas = []
    confirmed = []
    deaths = []
    recovered = []
    for location in json['locations']:
        timeline_entry = {}
        entry = {
            'id':location['id'],
            'lat':location['coordinates']['latitude'],
            'lon':location['coordinates']['longitude'],
            'Date':pandas.to_datetime(location['last_updated']),
            'province':location['province'],
            'country_code': location['country_code'],
            'country': location['country'] ,
            'cases':location['latest']['confirmed'],
            'deaths':location['latest']['deaths'],
            'recovered':location['latest']['recovered'],
            'county':''}
        if 'county' in location.keys():
            entry['county'] = location['county']
        if 'state' in location.keys():
            entry['state'] = location['state']
        entry['source'] = source
        for_pandas.append(entry)
        
        if 'timelines' in location.keys():
            for status in location['timelines']:
                for date in location['timelines'][status]['timeline']: 
                        sub_entry = {'id':location['id'],
                                     'Date':pandas.to_datetime(date),
                                     status:location['timelines'][status]['timeline'][date]}
                        if status ==  'confirmed':
                            confirmed.append(sub_entry)
                        elif status == 'deaths':
                            deaths.append(sub_entry)
                        else:
                            recovered.append(sub_entry)
    if confirmed:
        timeline_df = pandas.DataFrame(confirmed).merge(
            pandas.DataFrame(deaths),on=['id','Date']).merge(
            pandas.DataFrame(recovered), on=['id','Date'])
        main_df = pandas.DataFrame(for_pandas)
        timeline_df = timeline_df.merge(main_df[['id','lat','lon','province','country_code','country','source']],on=['id'])
        
        return main_df,timeline_df
    else:
        return pandas.DataFrame(for_pandas)
    
jhu_df,jhu_df_time = parse_api_json(covid19_jhu,'JHU')
csbs_df = parse_api_json(covid19_csbs,'CSBS')

In [ ]:
pd = pandas
date_mapper = pd.DataFrame(
    jhu_df_time['Date'].unique(), columns=['Date'])
date_mapper['Date_text'] = date_mapper['Date'].dt.strftime('%m/%d/%y')

In [ ]:
date_mapper.index[0]

In [ ]:
def per_x_cases(grouper,df):
    new_cases_by_country = []
    dates = date_mapper['Date']
    sub_group = df[df[grouper] != ""]
    groupers = sub_group[grouper].unique()

    for group in groupers:
        sub_country = sub_group[sub_group[grouper] == group]
        new_cases_by_country.append(
            {grouper: group, 'Date': dates[0],
             'New Cases': sub_country.loc[sub_country['Date'] == dates[0], 'confirmed'].sum(),
             'New Deaths': 0,
             'New Recovery': 0})
        for date_index in range(1, len(dates)):
            current_date = dates[date_index]
            day_before = dates[date_index-1]
            # print(current_date,day_before)
            t_c, t_d, t_r = sub_country.loc[sub_country['Date']
                                            == current_date, :].sum()[['confirmed', 'deaths', 'recovered']]

            y_c, y_d, y_r = sub_country.loc[sub_country['Date']
                                            == day_before, :].sum()[['confirmed', 'deaths', 'recovered']]

            new_cases = t_c - y_c
            new_deaths = t_d - y_d
            new_recovery = t_r - y_r
            new_cases_by_country.append(
                {grouper: group, 'Date': current_date, 'New Cases': new_cases,
                 'New Deaths': new_deaths, 'New Recovery': new_recovery})
    return pd.DataFrame(new_cases_by_country)

provence_df_per_day = per_x_cases('province',jhu_df_time)
country_df_per_day = per_x_cases('country',jhu_df_time)

In [ ]:
import sys
import warnings
import numpy as np
import pandas as pd
import argparse
import pprint
import plotly.graph_objects as go
import plotly.express as p

In [ ]:



# mapbox_style = "mapbox://styles/plotlymapbox/cjvprkf3t1kns1cqjxuxmwixz"
mapbox_style = 'dark'
mapbox_access_token = open('.mapbox_token').readlines()[0]

# # Import from S3:
# # merged_df = pd.read_csv(
# #     'https://jordansdatabucket.s3-us-west-2.amazonaws.com/covid19data/Merged_df.csv.gz', index_col=0).fillna('')
# # per_day_stats_by_state = pd.read_csv(
# #     'https://jordansdatabucket.s3-us-west-2.amazonaws.com/covid19data/per_day_stats_by_state.csv.gz', index_col=0)
# # per_day_stats_by_country = pd.read_csv(
# #     'https://jordansdatabucket.s3-us-west-2.amazonaws.com/covid19data/per_day_stats_by_country.csv.gz', index_col=0)
# # per_day_stats_by_county = pd.read_csv(
# #     'https://jordansdatabucket.s3-us-west-2.amazonaws.com/covid19data/per_day_stats_by_county.csv.gz', index_col=0)


# merged_df['Date'] = pd.to_datetime(merged_df['Date'])
# per_day_stats_by_state['Date'] = pd.to_datetime(
#     per_day_stats_by_state['Date']).fillna('')
# per_day_stats_by_country['Date'] = pd.to_datetime(
#     per_day_stats_by_country['Date']).fillna('')
# per_day_stats_by_county['Date'] = pd.to_datetime(
#     per_day_stats_by_county['Date']).fillna('')
date_mapper = pd.DataFrame(
    jhu_df_time['Date'].unique(), columns=['Date'])
date_mapper['Date_text'] = date_mapper['Date'].dt.strftime('%m/%d/%y')

min_date = min(date_mapper.index)
max_date = max(date_mapper.index)

# centroid_country_mapper = merged_df.groupby(
#     'Country/Region').apply(lambda x: x.sort_values('Cases')[::-1].iloc[0][['Lat', 'Long']])
# centroid_country_mapper = {x[0]: {'Long': x[1]['Long'], 'Lat': x[1]['Lat']}
#                            for x in centroid_country_mapper.iterrows()}


In [ ]:
jhu_df_time

In [ ]:
official_date = jhu_df_time.iloc[61]['Date']

In [ ]:
country_mapper = jhu_df_time[jhu_df_time['Date'] == official_date].groupby('country',as_index=False).apply(lambda x: x.sort_values('confirmed')[::-1].head(1)[['country','id','lat','lon']])

In [ ]:
country_mapper['country'].unique()

In [ ]:
sub_df =jhu_df_time[jhu_df_time['Date'] == official_date].groupby('country',as_index=False).sum().merge(country_mapper,on=['country'])

In [ ]:
jhu_df_time[jhu_df_time['Date'] == official_date].groupby('province',as_index=False).count().sort_values('id')

In [ ]:
jhu_df_time = jhu_df_time[jhu_df_time['province'].str.split(', ').str.len()==1]

In [ ]:
csbs_df['lat']= csbs_df['lat'].astype(float)
csbs_df['lon']= csbs_df['lon'].astype(float)

csbs_df.groupby(['county','state','lat','lon'],as_index=False).sum()

In [ ]:
csbs_df[csbs_df['Date'] == date_mapper.iloc[0]['Date']]

In [ ]:
def get_graph_state(date_int, group, metrics, figure):
    # Get initial zoom and shit if the figure is already drawn
    if not figure:
        lat = 15.74
        lon = -1.4
        zoom = 1.6
    elif "layout" in figure:
        lat = figure["layout"]["mapbox"]['center']["lat"]
        lon = figure["layout"]["mapbox"]['center']["lon"]
        zoom = figure["layout"]["mapbox"]["zoom"]

    if not figure:
        lat = 15.74
        lon = -1.4
        zoom = 1.6
    elif "layout" in figure:
        lat = figure["layout"]["mapbox"]['center']["lat"]
        lon = figure["layout"]["mapbox"]['center']["lon"]
        zoom = figure["layout"]["mapbox"]["zoom"]

    if 'cases' in metrics:
        normalizer = 'confirmed'
    elif 'active' in metrics:
        normalizer = 'Active'
    elif 'recovery' in metrics:
        normalizer = 'recovered'
    else:
        normalizer = 'deahts'


    official_date = date_mapper.iloc[date_int]['Date']
    # print(date_int, official_date)

    if group == 'country':
        country_mapper = jhu_df_time[
            jhu_df_time['Date'] == official_date].groupby(
            'country',as_index=False).apply(lambda x: x.sort_values('confirmed')[::-1].head(1)[['country','id','lat','lon']])
        sub_df =jhu_df_time[
            jhu_df_time['Date'] == official_date].groupby('country',as_index=False).sum().merge(country_mapper,on=['country'])
        sizeref = 2. * jhu_df_time.groupby(
            ['Date', group]).sum().max()[normalizer] / (20 ** 2)

    elif group == 'province':
        sub_df = jhu_df_time[jhu_df_time[group] != '']
        province_mapper = sub_df[
            sub_df['Date'] == official_date].groupby(
                group,as_index=False).apply(lambda x: x.sort_values('confirmed')[::-1].head(1)[[group,'id','lat','lon']])
        sub_df =sub_df[
            sub_df['Date'] == official_date].groupby(group,as_index=False).sum().merge(province_mapper,on=['province'])
        sizeref = 2. * jhu_df_time.groupby(
            ['Date', group]).sum().max()[normalizer] / (20 ** 2)

    
    elif group == 'county':
        sub_df = csbs_df.groupby(['county','state','lat','lon'],as_index=False).sum()        
        sub_df.rename({'cases':'confirmed'},axis=1,inplace=1)
        sizeref = 2. * sub_df.max()[normalizer] / (20 ** 2)


    sub_df['Active'] = sub_df['confirmed'] - sub_df['deaths'] - sub_df['recovered']
    sub_df['Text_Cases'] = sub_df[group] + '<br>Total Cases at {} : '.format(
        official_date.strftime('%m/%d/%y')) + sub_df['confirmed'].apply(lambda x: "{:,}".format(int(x)))
    sub_df['Text_Death'] = sub_df[group] + '<br>Total Deaths at {} : '.format(
        official_date.strftime('%m/%d/%y')) + sub_df['deaths'].apply(lambda x: "{:,}".format(int(x)))
    sub_df['Text_Recover'] = sub_df[group] + '<br>Total Recoveries at {} : '.format(
        official_date.strftime('%m/%d/%y')) + sub_df['recovered'].apply(lambda x: "{:,}".format(int(x)))

    sub_df['Text_Active'] = sub_df[group] + '<br>Total Active at {} : '.format(
        official_date.strftime('%m/%d/%y')) + sub_df['Active'].apply(lambda x: "{:,}".format(int(x)))

    fig = go.Figure()
    if 'cases' in metrics:
        fig.add_trace(go.Scattermapbox(
            lon=sub_df['lon'].astype(float) +
            np.random.normal(0, .02, len(sub_df['lon'])),
            lat=sub_df['lat'].astype(float) +
            np.random.normal(0, .02, len(sub_df['lat'])),
            customdata=sub_df[group],
            textposition='top right',
            text=sub_df['Text_Cases'],
            hoverinfo='text',
            mode='markers',
            name='cases',
            marker=dict(
                sizeref=sizeref,
                sizemin=3,
                size=sub_df['confirmed'],
                color='yellow')))

    if 'deaths' in metrics:
        fig.add_trace(go.Scattermapbox(
            lon=sub_df['Long'] +
            np.random.normal(0, .02, len(sub_df['Long'])),
            lat=sub_df['Lat'] +
            np.random.normal(0, .02, len(sub_df['Lat'])),
            customdata=sub_df[group],
            textposition='top right',
            text=sub_df['Text_Death'],
            hoverinfo='text',
            name='deaths',
            mode='markers',
            marker=dict(
                sizeref=sizeref,
                sizemin=3,
                size=sub_df['deaths'],
                color='red')))

    if 'recovery' in metrics:
        fig.add_trace(go.Scattermapbox(
            lon=sub_df['Long'] +
            np.random.normal(0, .02, len(sub_df['Long'])),
            lat=sub_df['Lat'] +
            np.random.normal(0, .02, len(sub_df['Lat'])),
            customdata=sub_df[group],
            textposition='top right',
            text=sub_df['Text_Recover'],
            hoverinfo='text',
            name='recoveries',
            mode='markers',
            marker=dict(
                sizeref=sizeref,
                sizemin=3,
                size=sub_df['recovered'],
                color='green')))
    if 'active' in metrics:
        fig.add_trace(go.Scattermapbox(
            lon=sub_df['Long'] +
            np.random.normal(0, .02, len(sub_df['Long'])),
            lat=sub_df['Lat'] +
            np.random.normal(0, .02, len(sub_df['Lat'])),
            customdata=sub_df[group],
            textposition='top right',
            text=sub_df['Text_Active'],
            hoverinfo='text',
            name='active',
            mode='markers',
            marker=dict(
                sizeref=sizeref,
                sizemin=3,
                size=sub_df['Active'],
                color='orange')))

    if not metrics:
        fig.add_trace(go.Scattermapbox(
            lon=[],
            lat=[]
        ))
    layout = dict(
        title_text='The Corona is Coming',
        autosize=True,
        showlegend=True,
        mapbox=dict(
            accesstoken=mapbox_access_token,
            style=mapbox_style,
            zoom=zoom,
            center=dict(lat=lat, lon=lon)
        ),
        hovermode="closest",
        margin=dict(r=0, l=0, t=0, b=0),
        dragmode="pan",
        legend=dict(
            x=0.92,
            y=1,
            traceorder="normal",
            font=dict(
                family="sans-serif",
                size=14,
                color="white"
            ),
            bgcolor='rgba(0,0,0,0)',
            # bordercolor="",
            # borderwidth=2
        )
    )

    fig.update_layout(layout)
    return fig

fig = get_graph_state(61, 'county', ['cases'],'')
fig

In [ ]:
print(fig)

In [ ]:
sub = state_df.merge(county_df,on=['Province/State'],suffixes=('','_county')) 
sub['Cases'] = sub['Cases'] + sub['Cases_county']
sub['Deaths'] = sub['Deaths'] + sub['Deaths_county']
sub['Recovery'] = sub['Deaths'] + sub['Recovery_county']

In [ ]:
sub

In [ ]:
lat = 15.74
lon = -1.4
zoom = 1.6
group = 'Province/State'
sizeref = 2. * merged_df.groupby(
        ['Date', group]).sum().max()['Cases'] / (20 ** 2)
fig = go.Figure()
fig.add_trace(go.Scattermapbox(
    lon=sub_df['Long'] +
    np.random.normal(0, .02, len(sub_df['Long'])),
    lat=sub_df['Lat'] +
    np.random.normal(0, .02, len(sub_df['Lat'])),
    customdata=sub_df[group],
    textposition='top right',
    text=sub_df['Text_Cases'],
    hoverinfo='text',
    mode='markers',
    marker=dict(
        sizeref=sizeref,
        sizemin=3,
        size=sub_df['Cases'],
        color='yellow')))
    
layout = dict(
    title_text='The Corona is Coming',
    autosize=True,
    showlegend=False,
    mapbox=dict(
        accesstoken=mapbox_access_token,
        style=mapbox_style,
        zoom=zoom,
        center=dict(lat=lat, lon=lon)
    ),
    #hovermode="closest",
    margin=dict(r=0, l=0, t=0, b=0),
    dragmode="pan",
)

fig.update_layout(layout)
fig

In [ ]:
list(sub_df['Cases'])